# Speed up

Convert the basic monthly CPS datafiles from 1998 to present into a sql database.


Updated: February 18, 2018 -- @bd_econ

In [9]:
# Preliminaries and data location
import pandas as pd
import os, re, struct, sqlite3

# Location of data
os.chdir('E:/08_Other/Archive/data')

# Location of database, if empty, will be created.
conn = sqlite3.connect('cps_test.db')

### Data Dictionaries

This section needs some series work!

In [25]:
# Read data dictionary and extract columns of interest
s = ['PWORWGT', 'PWCMPWGT', 'HRHHID', 'HURESPLI', 'HRLONGLK', 'HRHHID2', 
     'PRERNWA', 'PRERNHLY', 'PRTAGE', 'PEAGE', 'PTERNWA', 'PTERNHLY',]
s2 = ['HRMONTH', 'PESEX', 'PEMLR', 'PENLFRET', 'PENLFACT', 'PRDISC', 'GESTFIPS',
      'HRMIS', 'PRCOW1', 'PRFTLF', 'PREMPNOT', 'PRCIVLF', 'PEJHRSN', 'PEMJOT',
      'PEEDUCA', 'PRWKSTAT', 'PRDTOCC1', 'GTMETSTA', 'GEMETSTA']

# Regular expressions used to capture pattern in Census data dictionaries
p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\s+.*?(\d\d*).*?(\d\d+)')

# Read the data dictionary file to get the column names and locations
dd_open = open('jan03dd.txt', 'r', encoding='iso-8859-1').read()

# Regular expression for info of interest based on pattern p
dd = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd_open) if i[0] in (s+s2)]

# Manually fix up dictionary
dd[6] = ('GTMETSTA', 105, 1)
dd[7] = ('PRTAGE', 122, 2) 
dd[19] = ('PRDTOCC1', 476, 2)

In [26]:
# Use struct to read files faster 
def struct_constr(fieldspecs):
    """Specify which characters to retrieve and which to ignore"""
    unpack_len = 0
    unpack_fmt = ""
    for fieldspec in fieldspecs:
        start = fieldspec[1] - 1
        end = start + fieldspec[2]
        if start > unpack_len:
            unpack_fmt += str(start - unpack_len) + "x"
        unpack_fmt += str(end - start) + "s"
        unpack_len = end
    return struct.Struct(unpack_fmt).unpack_from

In [27]:
# Read single monthly file and convert to pandas dataframe
unpack = struct_constr(dd) 
row_list = []
with open('jan04pub.dat', 'r') as file:
    for line in file:
        row = tuple(map(int, unpack(line.encode())))
        if row[-1] > 0:  # Filter out weightless rows
            row_list.append(row)
    
df = (pd.DataFrame(row_list, columns=[v[0] for v in dd])
      .apply(pd.to_numeric, downcast='signed'))

In [13]:
# Export to different filetypes
df.to_sql('cps_oct16', conn, if_exists='replace')

In [14]:
df.to_csv('cps_oct16.csv')

In [22]:
df.to_pickle('cps_oct16.pkl')

In [24]:
df2 = pd.read_pickle('cps')

0         1
1         2
2         2
3         1
4         2
5         1
6         1
7         2
8         1
9         2
10        1
11        1
12        2
13        1
14        1
15        2
16        1
17        2
18        1
19        1
20        1
21        1
22        1
23        2
24        2
25        1
26        2
27        2
28        2
29        2
         ..
104732    2
104733    1
104734    2
104735    1
104736    2
104737    1
104738    2
104739    2
104740    2
104741    1
104742    1
104743    2
104744    1
104745    1
104746    2
104747    1
104748    2
104749    1
104750    2
104751    2
104752    2
104753    1
104754    2
104755    1
104756    2
104757    1
104758    2
104759    1
104760    2
104761    1
Name: PESEX, Length: 104762, dtype: int8

In [ ]:
col_specs = [(i[1], i[1]+i[2]-1) for i in d[2016]]
names = [i[0] for i in d[2016]]

In [ ]:
df = pd.read_fwf('data/oct16pub.dat', header=None, colspecs=col_specs, names=names)

In [ ]:
df.memory_usage(index=True).sum()

In [ ]:
l = []
unpk = struct_constr(fieldspecs) 
with open('data/oct16pub.dat', 'r') as f:
    for line in f:       
        [int(x) for x in unpacker(line.encode())
        if int(raw_fields[-1]) > 0:   # Filter out rows with no weight
            l.append(tuple(int(x) for x in raw_fields))
            
df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]).apply(
    pd.to_numeric, downcast='unsigned')

In [ ]:
with open('data/oct16pub.dat', 'r') as f:
    fields = unpacker(line.encode())
    df = pd.DataFrame([[int(x) for x in unpacker(line.encode())] 
                    for line in f if int(unpacker(line.encode())[-1]) > 0], 
                  columns = [f[0] for f in fieldspecs])
#    for col in [f[0] for f in fieldspecs if f[0] in s2]: 
#        df[col] = df[col].astype('category').cat.as_ordered()
    df = df.apply(pd.to_numeric, downcast='unsigned')

In [ ]:
#[f[0] for f in fieldspecs if f[0] in s2]

In [ ]:
#fieldspecs

In [ ]:
unpack = struct_constr(fieldspecs) 
row_list = []
with open('data/oct16pub.dat', 'r') as file:
    for line in file:
        row = tuple(map(int, unpack(line.encode())))
        if row[-1] > 0:  # Filter out weightless rows
            row_list.append(r)
            
            
#    d = [[int(s) for s in unpack(r.encode())] for r in f if int(unpack(r.encode())[-1]) > 0]
    
df = (pd.DataFrame(d, columns=[f[0] for f in fieldspecs])
          .apply(pd.to_numeric, downcast='signed'))

In [ ]:
unpack = struct_constr(fieldspecs) 
d = []
with open('data/oct16pub.dat', 'r') as file:
    for line in [f for f in file if tuple(map(int, unpack(f.encode())))[-1] > 0]:
        

In [ ]:
tuple(map(int, unpack(line.encode())))[-1]

In [ ]:
tuple(int(s) for s in unpack(line.encode()))

In [ ]:
mem_usage(df['PRDTOCC1'].replace(-1, np.nan).apply(pd.to_numeric, downcast='unsigned'))#astype('uint16')

In [ ]:
mem_usage(df['PRDTOCC1'])#.replace(-1, np.nan)

In [ ]:
print(mem_usage(df))

In [ ]:
len(df)

In [ ]:
df.memory_usage(index=True).sum()

In [ ]:
#df.info(memory_usage='deep')

### Tools for manual use

In [ ]:
# Tools for manual use, as needed

# Read a data dictionary in the notebook
#print(open(f'data/jan10dd.txt', 'r', encoding='iso-8859-1').read())

# Unzip files
#from zipfile import ZipFile
#for file in [f for f in os.listdir('data/') if f.endswith('pub.zip')]:
#    with ZipFile(f'data/{file}', 'r') as zip_ref:
#        zip_ref.extractall('data/')

# Convert .cps file extension into .dat (early data files end in .cps)
#for file in [f for f in os.listdir('data/') if f.endswith('cps')]:
#    os.rename(f'data/{file}', f'data/{file[:-4]}.dat')

# Drop table from database
#c = conn.cursor()
#c.execute("DROP TABLE cps_2012")
#conn.commit()
#conn.close()

### Functions for reading the data

In [ ]:
# Set of functions for parsing raw data

# Use struct to read files faster 
def struct_constr(fieldspecs):
    """Build a fast way to read fixed width files"""
    unpack_len = 0
    unpack_fmt = ""
    for fieldspec in fieldspecs:
        start = fieldspec[1] - 1
        end = start + fieldspec[2]
        if start > unpack_len:
            unpack_fmt += str(start - unpack_len) + "x"
        unpack_fmt += str(end - start) + "s"
        unpack_len = end
    field_indices = range(len(fieldspecs))
    return struct.Struct(unpack_fmt).unpack_from

# Convert valid lines to list
def fwf_to_list(file, unpacker, l):
    with open(f'data/{file}', 'r', encoding='utf-8') as f:
        for line in f:
            raw_fields = unpacker(line.encode())
            if int(raw_fields[-1]) > 0:   # Filter out rows with no weight
                l.append(tuple(int(x) for x in raw_fields))

# Convert list of lists to pandas df
def list_to_df(l, fieldspecs, year):
    """Store list as pandas dataframe"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    return df

# Send pandas df to SQL
def df_to_sql(df, year):
    """Send pandas df to sql database"""
    df.to_sql(f'cps_{year}', conn, if_exists='replace')

# Previous two, combined
def list_to_sql(l, fieldspecs, year):
    """Store list into sql database"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    df.to_sql(f'cps_{year}', conn, if_exists='replace')

# This is source of problem with 2004 and 2012
def special_years(year, fs1, fs2, sm):
    """Handle cases where dictionary is split in middle of year.
       Takes two sets of field specifications, and the split month"""
    if year == year:
        fieldspecs = fs1
        unpacker = struct_constr(fs1)

        l = []
        # Fill list with monthly data from each monthly file
        for file in [f'{calendar.month_name[i].lower()[:3]}{str(year)[2:]}pub.dat' for i in range(1,sm)]:
            fwf_to_list(file, unpacker, l)
        df2 = list_to_df(l, fs1, year)
                        
        fieldspecs = fs2
        unpacker = struct_constr(fs2)
        
        l = []
        # Fill list with monthly data from each monthly file
        for file in [f'{calendar.month_name[i].lower()[:3]}{str(year)[2:]}pub.dat' for i in range(sm,13)]:
            fwf_to_list(file, unpacker, l)
        df3 = list_to_df(l, fs2, year)
        df4 = df2.append(df3)
        df_to_sql(df4, year)

# Manages the other functions
def monthly_to_annual(year):
    """Read monthly files and store as one annual file"""
    # Field specs (field name, start pos (1-relative), len)
    if year not in [2004, 2005, 2012]:
        fieldspecs = d[year]
        unpacker = struct_constr(fieldspecs)

        l = []
        # Fill list with monthly data from each monthly file
        for file in [f for f in os.listdir('data/') if f.endswith(f'{str(year)[-2:]}pub.dat')]:
            fwf_to_list(file, unpacker, l)
        list_to_sql(l, fieldspecs, year)
        
    if year == 2012: special_years(2012, d[2011], d['may12'], 5)
    if year == 2005: special_years(2005, d['may04'], d['augnov05'], 8)
    if year == 2004: special_years(2004, d['jan03'], d['may04'], 5)

### Applying the function to selected years

In [ ]:
#for year in range(1998, 2019):
#    monthly_to_annual(year)

monthly_to_annual(2004)

In [ ]:
#df = pd.read_sql("SELECT * FROM cps_2004", conn)#.drop_duplicates())